# DPA Experiment

In [53]:
import sys
import tensorflow as tf

sys.path.append('./cs16')
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
from keras.callbacks import EarlyStopping
from keras.layers import (Activation, Attention, Bidirectional, Concatenate, Conv1D,
                           Dense, Dropout, Embedding, Flatten, GlobalMaxPooling1D,
                           Input, Layer, LSTM, MaxPooling1D, Multiply, Permute,
                           RepeatVector, Reshape, SpatialDropout1D, TimeDistributed)
from keras.models import Model
from keras import backend as K
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from keras import regularizers

import cs16.prep as prep16
import cs16.plot as plot16
import cs16.build as build16
imagesize = 64

import pandas as pd

data_type = input("Enter '1. single' for single dataset or '2. multi' for multiple dataset: ")

if data_type == '1':
    file_path = 'single.txt'
    folder_path = './data/MVSA/single/'
elif data_type == '2':
    file_path = 'multi.txt'
    folder_path = './data/MVSA/multiple/'
else:
    print("Invalid input. Please enter either 'single' or 'multi'.")
    exit()

df = pd.read_csv(file_path, index_col=None, encoding='ISO-8859-1')

Enter '1. single' for single dataset or '2. multi' for multiple dataset: 2


In [54]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import pandas as pd
import matplotlib.pyplot as plt
# Download stopwords and punkt tokenizer if not already downloaded
nltk.download('stopwords')
nltk.download('punkt')

# Define a function to preprocess text
def nlp_text(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Remove URLs using regex
    text = re.sub(r'http\S+', '', text)
    
    # Remove punctuation using regex
    text = re.sub(r'[^\w\s]', '', text)
    
    # Tokenize text into individual words
    words = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    
    # Stem words using Porter Stemmer
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]
    
    # Join words back into a single string
    text = ' '.join(words)
    
    return text

# Apply the preprocess_text function to the 'tweet' column of the dataframe
df['tweet'] = df['tweet'].apply(nlp_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ausco\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ausco\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [55]:
X_text, y_text = prep16.preprocess_text(df)
X_train_text, X_val_text, X_test_text, \
y_train_text, y_val_text, y_test_text = prep16.split_data(X_text, y_text, random_state=42)

X_polar, y_polar = prep16.preprocess_text(df,label = 'polarity')
X_train_polar, X_val_polar, X_test_polar, \
y_train_polar, y_val_polar, y_test_polar = prep16.split_data(X_polar, y_polar, random_state=42)

#image_data_s, image_label_s = prep16.preprocess_images(df, folder_path, imagesize)
#y_s = to_categorical(image_label_s, num_classes=3)

#X_train_image, X_val_image, X_test_image, \
#y_train_image, y_val_image, y_test_image= prep16.split_data(image_data_s, y_s, random_state=42)

# text only......
y_train = to_categorical(y_train_text, num_classes=3)
y_val =to_categorical(y_val_text, num_classes=3)
y_test =to_categorical(y_test_text, num_classes=3)
print(f"dataset processed is {file_path}")

dataset processed is multi.txt


# LSTM

In [67]:
from keras.layers import LSTM
#=============================================
# LSTM
#=============================================
# Text model
text_af = 'tanh'
text_input = Input(shape=(100,), name='text_input')
lstm1 = Embedding(input_dim=10000, output_dim=32)(text_input)
lstm1 = Dropout(0.3)(lstm1)  # Add a dropout layer
lstm1 = LSTM(128, activation=text_af)(lstm1)
lstm1 = Dense(64, activation=text_af, kernel_regularizer=regularizers.l2(0.01))(lstm1)
lstm1 = Dropout(0.5)(lstm1)  # Add a dropout layer
text_output = Dense(3, activation='sigmoid', kernel_regularizer=regularizers.l2(0.01))(lstm1)
textclassifier = 'LSTM'
print(f"{textclassifier}")

LSTM


# LSTM + ATT

In [64]:
class ContentAttention(Layer):
    def __init__(self, **kwargs):
        super(ContentAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1), initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1), initializer="zeros")
        super(ContentAttention, self).build(input_shape)

    def call(self, x):
        e = K.tanh(K.dot(x, self.W) + self.b)
        a = K.softmax(e, axis=1)
        output = x * a
        return K.sum(output, axis=1) 
from keras.layers import LSTM
#=============================================
# LSTM
#=============================================
# Text model
text_af = 'tanh'
text_input = Input(shape=(100,), name='text_input')
lstm1 = Embedding(input_dim=10000, output_dim=32)(text_input)
lstm1 = Dropout(0.3)(lstm1)  # Add a dropout layer
lstm1 = LSTM(256, activation=text_af)(lstm1)
lstm1 = Reshape((1, 256))(lstm1)
lstm1 = ContentAttention()(lstm1)
lstm1 = Dense(64, activation=text_af, kernel_regularizer=regularizers.l2(0.01))(lstm1)
lstm1 = Dropout(0.5)(lstm1)  # Add a dropout layer
text_output = Dense(3, activation='sigmoid', kernel_regularizer=regularizers.l2(0.01))(lstm1)
textclassifier = 'LSTM +ATT'
print(f"{textclassifier}")    

LSTM +ATT


# BiLSTM

In [72]:
from keras.layers import LSTM
#=============================================
# LSTM
#=============================================
# Text model
text_af = 'tanh'
text_input = Input(shape=(100,), name='text_input')
t = Embedding(input_dim=10000, output_dim=32)(text_input)
t = Dropout(0.3)(t)  # Add a dropout layer
t = Bidirectional(LSTM(128, activation=text_af))(t)
t = Dense(64, activation=text_af, kernel_regularizer=regularizers.l2(0.01))(t)
t = Dropout(0.5)(t)  # Add a dropout layer
text_output = Dense(3, activation='sigmoid', kernel_regularizer=regularizers.l2(0.01))(t)
textclassifier = 'Bi-LSTM'
print(f"{textclassifier}")

Bi-LSTM


# DUAL

In [69]:
from keras.layers import LSTM
#=============================================
# LSTM
#=============================================
# Text model
text_af = 'tanh'
text_input = Input(shape=(100,), name='text_input')

#----------LSTM1----------------
lstm1 = Embedding(input_dim=10000, output_dim=32)(text_input)
lstm1 = Dropout(0.3)(lstm1)  # Add a dropout layer
lstm1 = LSTM(128, activation=text_af)(lstm1)
lstm1 = Dense(64, activation=text_af, kernel_regularizer=regularizers.l2(0.03))(lstm1)
lstm1 = Dropout(0.5)(lstm1)  # Add a dropout layer

#---------LSTM2-----------------
lstm2 = Embedding(input_dim=10000, output_dim=32)(text_input)
lstm2 = Dropout(0.3)(lstm2)  # Add a dropout layer
lstm2 = LSTM(64, activation=text_af)(lstm2)
lstm2 = Dense(32, activation=text_af, kernel_regularizer=regularizers.l2(0.01))(lstm2)
lstm2 = Dropout(0.5)(lstm2)  # Add a dropout layer

#----------local fusion---------------
merged_layer = Concatenate()([lstm1, lstm2]) 
merged = Dense(3, activation='softmax')(merged_layer)
#merged = Reshape((1, 128))(merged)  
#merged = Bidirectional(LSTM(64))(merged)
#text_output = Dense(3, activation='sigmoid', kernel_regularizer=regularizers.l2(0.01))(merged)
text_output = merged
textclassifier = 'Dual LSTM'
print(f"{textclassifier}")

Dual LSTM


# Function

In [51]:
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

def print_classification_report(model, X_test, y_test):
    # Generate the predicted labels
    y_pred = model.predict(X_test)

    # Convert the predicted probabilities to class labels
    y_pred_labels = np.argmax(y_pred, axis=1)
    y_true_labels = np.argmax(y_test, axis=1) 

    # Generate the classification report
    report = classification_report(y_true_labels, y_pred_labels, digits=4)
    print(report)


# RUN MODEL

In [73]:
import tensorflow as tf
# Define the model
model = Model(inputs=text_input, outputs=text_output)
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model
model.fit(X_train_text, y_train, validation_data=(X_val_text, y_val), epochs=10, batch_size=128)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test_text, y_test)
print(f' {textclassifier} Test accuracy:{test_acc}')
print_classification_report(model, X_test_text, y_test)

Epoch 1/10
122/122 [==============================] - 139s 1s/step - loss: 1.2219 - accuracy: 0.7115 - val_loss: 0.8125 - val_accuracy: 0.7360
Epoch 2/10
122/122 [==============================] - 146s 1s/step - loss: 0.7742 - accuracy: 0.7169 - val_loss: 0.7044 - val_accuracy: 0.7381
Epoch 3/10
122/122 [==============================] - 145s 1s/step - loss: 0.6638 - accuracy: 0.7364 - val_loss: 0.6996 - val_accuracy: 0.7365
Epoch 4/10
122/122 [==============================] - 151s 1s/step - loss: 0.6080 - accuracy: 0.7569 - val_loss: 0.7218 - val_accuracy: 0.7206
Epoch 5/10
122/122 [==============================] - 165s 1s/step - loss: 0.5705 - accuracy: 0.7749 - val_loss: 0.7554 - val_accuracy: 0.7149
Epoch 6/10
122/122 [==============================] - 169s 1s/step - loss: 0.5362 - accuracy: 0.7887 - val_loss: 0.8169 - val_accuracy: 0.7134
Epoch 7/10
122/122 [==============================] - 169s 1s/step - loss: 0.5062 - accuracy: 0.8047 - val_loss: 0.8226 - val_accuracy: 0.7072

In [71]:
# Duplicate a copy
print(f"Copy of >>>>>>>>>{textclassifier}>>>>>>>>>>>>>")
print_classification_report(model, X_test_text, y_test)

Copy of >>>>>>>>>Dual LSTM>>>>>>>>>>>>>
61/61 [==============================] - 4s 71ms/step
              precision    recall  f1-score   support

           0     0.2683    0.1769    0.2132       373
           1     0.7592    0.8534    0.8036      1378
           2     0.3245    0.2513    0.2832       195

    accuracy                         0.6634      1946
   macro avg     0.4507    0.4272    0.4333      1946
weighted avg     0.6215    0.6634    0.6383      1946



In [66]:
# Duplicate a copy
print(f"Copy of >>>>>>>>>{textclassifier}>>>>>>>>>>>>>")
print_classification_report(model, X_test_text, y_test)

Copy of >>>>>>>>>LSTM +ATT>>>>>>>>>>>>>
61/61 [==============================] - 8s 128ms/step
              precision    recall  f1-score   support

           0     0.2467    0.1984    0.2199       373
           1     0.7568    0.8628    0.8064      1378
           2     0.3467    0.1333    0.1926       195

    accuracy                         0.6624      1946
   macro avg     0.4501    0.3982    0.4063      1946
weighted avg     0.6180    0.6624    0.6325      1946

